# Installing required packages

In [ ]:
!pip install crewai
!pip install crewai crewai-tools



# Import necessary libraries and setting up environment variables

In [ ]:
from crewai import Crew, Task, Agent, LLM
from crewai.process import Process
import os

In [ ]:
os.environ['WATSONX_APIKEY']  = "<YOUR API KEY>"
os.environ['WATSONX_PROJECT_ID'] = "<YOUR PROJECT ID>"
os.environ['WX_URL']  = "<YOUR URL>"

# Defining LLM for the Loyalty Specialist Agent

In [ ]:
# Create the first LLM
llm = LLM(
    model="watsonx/meta-llama/llama-3-405b-instruct",
    base_url=os.environ.get("WX_URL"),
    project_id=os.environ.get("WATSONX_PROJECT_ID"),
    api_key=os.environ['WATSONX_APIKEY'],
    max_tokens=2000,
    temperature=0.7,
)

# Creating Agents and Tasks

In [ ]:
# Define the Loyalty Specialist Agent
loyalty_program_expert = Agent(
    llm=llm,
    role="Loyalty Specialist",
    goal="Offer the best and most relevant deals on credit cards and memberships to maximize customer engagement and retention by tailoring offers to the customer convestation-history : {convestation-history}.",
    backstory="You specialize in loyalty programs and personalized offers for Lendyr's customers. "
              "You analyze the convestation-history abd recommend the most suitable loyalty programs that provide maximum value. You aim to enhance customer loyalty by offering benefits that resonate with the customer's "
              "needs, including reward points, cashback, travel perks, and exclusive membership privileges.",
    allow_delegation=False,
    verbose=True,
)

# Define the Loyalty Program Recommendation Task
recommend = Task(
    description=(
        "1. Analyze the convestation-history with the customer to understand their preferences.\n"
        "2. Match the customer with the most relevant loyalty program offers, focusing on:\n"
        "   - Exclusive discounts and rewards matching their lifestyle.\n"
        "   - Cashback and points maximization tailored to their spending habits.\n"
        "   - Perks like free delivery or travel rewards for frequent buyers or travelers.\n"
        "   - Membership upgrades and special privileges for loyal customers.\n"
        "3. Highlight the benefits of higher loyalty tiers:\n"
        "   - Bronze: Entry-level perks and occasional discounts.\n"
        "   - Silver: Exclusive offers and faster reward accumulation.\n"
        "   - Gold: Premium benefits, VIP access, and enhanced services.\n"
        "4. Provide actionable insights on maximizing program value."
    ),
    expected_output="A concise list of personalized loyalty program benefits, with recommendations on how to maximize value.",
    agent=loyalty_program_expert,
)


writer = Agent(
    llm=llm,
    role="Writer",
    goal="Respond to conversations in a friendly, engaging tone while naturally promoting the best loyalty program benefits. Focus on the conversation history to recommend the most relevant loyalty program from Lendyr Bank.",
    backstory="You are a conversational expert specializing in loyalty programs and customer engagement. Your goal is to respond in a casual and informative manner, providing clear answers while subtly highlighting the benefits of Lendyr's offerings.",
    allow_delegation=False,
    verbose=True,
)
write = Task(
    description=(
        "1. Respond to the customer in a natural, conversational tone.\n"
        "2. Seamlessly mention relevant Credit Card and Membership benefits, linking them to the conversation history.\n"
        "3. Maintain a friendly, approachable, and engaging tone, incorporating emojis where appropriate to keep the conversation casual.\n"
        "4. Do not add any salutation or signature.\n"
        "5. Make sure the response is coherent wtih {convestation-history}.\n"
    ),
    expected_output="A friendly, conversational response that fits the conversation history. The response should naturally continue the flow of the conversation, highlighting relevant benefits without being too promotional.",
    agent=writer,
)


# Creating the Crew with Loyalty Program Expert and Writer agent

In [ ]:
# Define the Crew with all agents and tasks
crew = Crew(
    agents=[loyalty_program_expert, writer],
    tasks=[recommend, write],
)

In [ ]:
# Kick off the process with a travel-focused customer profile input and a question about perks
input_history_1 = {
        "convestation-history": """
        Leo:  
        I am planning a trip to Hawaii in July with my spouse, and we’re really excited about it! We’re hoping to spend around $7,000 to cover flights, accommodations, and activities. What loan options do you have for something like this?
        
        Bank Agent:  
        That sounds like an amazing trip! With your budget, we can look at a couple of options—either a personal loan or a travel loan. A personal loan offers more flexibility with repayment, while a travel loan is specifically designed for vacations like yours. Both options offer quick approval, and the interest rates will depend on your credit score. Would you like more details on each?
        
        Leo:  
        Yes, I’d love to hear more about the travel loan. I want to make sure we get the best deal for our trip.
        
        Bank Agent:  
        The travel loan is a great option for trips like yours. You can use it to cover everything—flights, hotels, activities, and even excursions—with low fixed interest rates. You’ll also have the flexibility to choose your repayment terms, anywhere from 12 to 36 months. The application process is simple, and you could get approved within 24 hours. Does that sound like a good fit for you and your spouse?
        
        Leo:  
        That sounds perfect. I think the travel loan is the way to go. What about booking the trip? We’re thinking about traveling in July.
        """
    }
crew.kickoff(
    inputs=input_history_1
)

# Lets Try with different inputs

In [ ]:
# Kick off the process with a travel-focused customer profile input and a question about perks
input_history_2 = {
        "convestation-history": """
        Leo:
        We’re planning a family trip to Lake Tahoe this spring with my spouse, our three kids, and a baby. We’re aiming to spend around $8,000 for flights, accommodations, and activities. What loan options do you have for something like this?
        
        Bank Agent:
        That sounds like a fun trip! With your budget, we can explore either a personal loan or a travel loan. A personal loan gives you more flexibility with repayment, while a travel loan is specifically designed for vacations like yours. Both options have quick approval, and the interest rates depend on your credit score. Would you like more details on each?
        
        Leo:
        Yes, please! I’d love to hear more about the travel loan. We want to make sure we’re getting the best deal for the trip.
        
        Bank Agent:
        The travel loan would be a great fit for your trip to Lake Tahoe. You can use it to cover everything—flights, accommodations, activities, and even baby-friendly extras—with low, fixed interest rates. You can also choose repayment terms from 12 to 36 months. The application process is easy, and in most cases, you can get approved within 24 hours. Does that sound like a good option for you and your family?
        
        Leo:
        That sounds great! I think the travel loan is the way to go. What about booking the trip itself? We’re thinking of going in the spring.
        
        """
    }
crew.kickoff(
    inputs=input_history_2
)

In [ ]:
# Kick off the process with a travel-focused customer profile input and a question about perks
input_history_3 = {
        "convestation-history": """
        Leo:  
        We’re planning a trip to Mount Hood this spring with my spouse and our puppy. We’re aiming to spend about $5,000 to cover our travel, accommodations, and activities. What loan options do you have for something like this?
        
        Bank Agent:  
        That sounds like a wonderful trip! With your budget, we can consider either a personal loan or a travel loan. A personal loan gives you more flexibility with repayment, while a travel loan is specifically designed for vacations like yours. Both options offer quick approval, and the interest rates will depend on your credit score. Would you like more details on each?
        
        Leo:  
        Yes, I’d love to know more about the travel loan. I want to make sure we get the best deal for our trip with our puppy.
        
        Bank Agent:  
        The travel loan is a perfect fit for trips like yours. You can use it to cover everything—flights, accommodations, activities, and even pet-friendly extras—with low fixed interest rates. Plus, you’ll have the flexibility to choose repayment terms from 12 to 36 months. The application process is quick, and you could be approved within 24 hours. Does that sound good for you, your spouse, and your puppy?
        
        Leo:  
        That sounds great! I think the travel loan would work well for us. What about booking the trip? We’re looking to go in the spring.
        """
    }
crew.kickoff(
    inputs=input_history_3
)

# Can you further enhance the crew to generate a more conversational-style output?